# Ingest Higher Educational Institutions for fact_employability

### Config

In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [2]:
remote_path = 'http://pacha.datawheel.us/education/empleabilidad/'
local_path = '../data/'

### Imports

In [3]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

In [4]:
df = download_file(remote_path+'ids/',local_path,'listado_instituciones.csv')
df = df.astype({'institution_id':'int','institution_group_id':'int','institution_subgroup_id':'int'})
list(df)

Downloading... http://pacha.datawheel.us/education/empleabilidad/ids/listado_instituciones.csv
Encoding: utf-8


['institution',
 'institution_id',
 'institution_group_name_es',
 'institution_group_name',
 'institution_group_id',
 'institution_subgroup_id',
 'institution_subgroup_name_es',
 'institution_subgroup_name']

### Ingest

In [5]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'education', 'dim_higher_educational_institutions')

DROP TABLE IF EXISTS education.dim_higher_educational_institutions;
CREATE TABLE "education"."dim_higher_educational_institutions" (
"institution" TEXT,
  "institution_id" INTEGER,
  "institution_group_name_es" TEXT,
  "institution_group_name" TEXT,
  "institution_group_id" INTEGER,
  "institution_subgroup_id" INTEGER,
  "institution_subgroup_name_es" TEXT,
  "institution_subgroup_name" TEXT
)
COPY "education"."dim_higher_educational_institutions" ("institution","institution_id","institution_group_name_es","institution_group_name","institution_group_id","institution_subgroup_id","institution_subgroup_name_es","institution_subgroup_name") FROM STDIN WITH CSV HEADER DELIMITER ',';
